In [1]:
import datasets
import matplotlib.pyplot as plt
import numpy as np
import nltk
import torch
import time
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [2]:
DATASET_DIR = './data'

esnli = datasets.load_dataset("esnli")

Found cached dataset esnli (/Users/lorenzorota/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc)


  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
esnli["train"].shape[0]

549367

In [8]:
SUBSET_SIZE = 0.1

In [10]:
dataset = esnli

In [15]:
        train_size = int(dataset['train'].shape[0] * SUBSET_SIZE)
        val_size = int(dataset['validation'].shape[0] * SUBSET_SIZE)
        test_size = int(dataset['test'].shape[0] * SUBSET_SIZE)

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'explanation_1', 'explanation_2', 'explanation_3'],
        num_rows: 549367
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'explanation_1', 'explanation_2', 'explanation_3'],
        num_rows: 9842
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'explanation_1', 'explanation_2', 'explanation_3'],
        num_rows: 9824
    })
})

In [24]:
t = datasets.DatasetDict()

In [26]:
        dataset2 = datasets.DatasetDict()
        dataset2['train'] = esnli['train'].shuffle(seed=42).select(range(train_size))
        dataset2['validation'] = esnli['validation'].shuffle(seed=42).select(range(val_size))
        dataset2['test'] = esnli['test'].shuffle(seed=42).select(range(test_size))

Loading cached shuffled indices for dataset at /Users/lorenzorota/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc/cache-175c704abdaa3ce8.arrow
Loading cached shuffled indices for dataset at /Users/lorenzorota/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc/cache-cfdf5e8fa5487187.arrow
Loading cached shuffled indices for dataset at /Users/lorenzorota/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc/cache-16f4f11d0f60d6b3.arrow


In [29]:
dataset2['train'][0]

{'premise': 'A group of people riding a yellow roller coaster.',
 'hypothesis': 'A group of people are riding a roller coaster.',
 'label': 0,
 'explanation_1': 'group of people riding a roller coaster. is same as group of people riding a yellow roller coaster',
 'explanation_2': '',
 'explanation_3': ''}

In [54]:
dataset2['train'].shuffle(seed=None)[0]

{'premise': 'A man holding a fishing rod stands nearby four men with a jet ski.',
 'hypothesis': 'One man is fishing.',
 'label': 1,
 'explanation_1': 'A man holding a fishing rod is not necessarily fishing.',
 'explanation_2': '',
 'explanation_3': ''}

In [57]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]

In [61]:
if torch.backends.mps.is_available()

True

In [65]:
torch.cuda.is_available()

False

In [67]:
torch.device("mps")

device(type='mps')

In [70]:
def select_device(choice='gpu'):
    """Return GPU device if available, else return CPU."""

    # Prioritize CUDA
    if choice=='gpu':
        if torch.cuda.is_available():
            device = torch.device('cuda')
        elif torch.backends.mps.is_available():
            device = torch.device('mps')
        else: # fall-back option
            device = torch.device('cpu')
    else:
        device = torch.device('cpu')
        
    return device

In [79]:
print(str(select_device())=='mps')

True
